In [ ]:
!pip install exifread opencv-python-headless

In [ ]:
from google.colab import files
from PIL import Image
import io

# Upload image
uploaded = files.upload()

# Load the uploaded image
image = None
for fn in uploaded.keys():
    image = Image.open(io.BytesIO(uploaded[fn]))
    print(f"Loaded image: {fn}")

# Display the image
image.show()

In [ ]:
!pip install exifread opencv-python-headless

In [ ]:
from google.colab import files
from PIL import Image
import io

# Upload image
uploaded = files.upload()

# Load the uploaded image
image = None
for fn in uploaded.keys():
    image = Image.open(io.BytesIO(uploaded[fn]))
    print(f"Loaded image: {fn}")

# Display the image
image.show()

In [2]:
# Install necessary packages in one Colab cell
!pip install ultralytics piexif geopy
!apt-get install -y exiftool

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libimage-exiftool-perl' instead of 'exiftool'
libimage-exiftool-perl is already the newest version (12.40+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import os

# List files in the root of Google Drive
os.listdir('/content/drive/My Drive/')

['samenvatting v4 stof scheikunde 16-17.pdf',
 'Discrete Mathematics and Its Applications.pdf',
 'Computer_Organization_and_Design_By_David_Patterson_5th_ed.pdf',
 'The 19th century Orthodox response to assimilation and progress.gdoc',
 'KAVV Jodendom.gdoc',
 'Work.pdf',
 'Work.gsheet',
 'Ecuador.gsheet',
 'SSO Assignment1.gdoc',
 'FDSv2.gdoc',
 'SSO Assingment2.gsheet',
 'Assingment 3 SSO.gsheet',
 'DS Project Text.gdoc',
 'Colab Notebooks',
 'feather-in-focus',
 'Societal Complexity and Designing with Data',
 'Group3-midterm.pdf',
 'Full_table_all_matches_for_dupes_DEC.csv',
 'Untitled document.gdoc',
 'Untitled spreadsheet.gsheet',
 'Japan Plan.pdf',
 'Brief papa.gdoc',
 'Japan Plan.gdoc',
 'Motivatie brief woo.gdoc',
 'Motivatie brief woo.pdf',
 'Elephants',
 'Brief papa.pdf',
 'AnimalData',
 'PhD referal Letter.gdoc',
 'PhD referal Letter.docx',
 'datasets',
 'Personal statement .gdoc',
 'Plymouth Personal Statement Letter.gdoc',
 'PHD Applications answers.gdoc',
 'Google AI Studi

In [5]:
cd drive/MyDrive/datasets/GPS

/content/drive/MyDrive/datasets/GPS


In [6]:
ls

 best_xl.pt      'GPS Test.ipynb'                 Processed_Distances_Output/
 Detections/      Images/                         Processed_Images_Output/
 DJI_0395-1.SRT   Processed_Coordinates_Output/


In [7]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO, SAM
import pandas as pd
import time
import os
from pathlib import Path
import matplotlib.pyplot as plt
from pyproj import Transformer
import math
import re
from datetime import datetime, timedelta
import subprocess
import json
import csv

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [16]:
def Find_Elephants(image_path):
  # Load the YOLOv11 model
  model = YOLO("best_xl.pt")  # Replace with your model path

  # Input image path
  results = model.predict(source=image_path, conf=0.6)

  # Prepare output folder
  output_dir = "Detections"
  os.makedirs(output_dir, exist_ok=True)

  # Load image
  im1 = cv2.imread(image_path)
  height, width, _ = im1.shape

  # Get base filename (without extension)
  base_name = os.path.splitext(os.path.basename(image_path))[0]

  # Prepare output data and counter
  yolo_format_data = []
  num_objects = 0  # Counter for detected objects

  # Process each detected box
  for box in results[0].boxes:
      num_objects += 1

      cls = int(box.cls[0])
      conf = float(box.conf[0])
      x1, y1, x2, y2 = map(float, box.xyxy[0])

      # Draw bounding box and label on image
      cv2.rectangle(im1, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
      cv2.putText(im1, f'Class: {cls}, Conf: {conf:.2f}', (int(x1), int(y1) - 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

      # Normalize xmin, ymin, xmax, ymax
      xmin_norm = x1 / width
      ymin_norm = y1 / height
      xmax_norm = x2 / width
      ymax_norm = y2 / height

      # Append to YOLO format list with normalized coordinates
      yolo_format_data.append(f"{cls} {conf:.6f} {xmin_norm:.6f} {ymin_norm:.6f} {xmax_norm:.6f} {ymax_norm:.6f}")

  # Save annotated image and txt file to Detections folder
  output_image_path = os.path.join(output_dir, f"{base_name}.jpg")
  output_txt_path = os.path.join(output_dir, f"{base_name}.txt")

  cv2.imwrite(output_image_path, im1)
  with open(output_txt_path, 'w') as f:
      f.write("\n".join(yolo_format_data))

In [25]:
def extract_exiftool_metadata(image_path):
    # Run exiftool and get JSON output
    result = subprocess.run(
        ['exiftool', '-j', image_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    if result.returncode != 0:
        raise RuntimeError(f"exiftool failed: {result.stderr}")

    metadata = json.loads(result.stdout)[0]
    return metadata

def extract_exif_metadata_partial(image_path):
    metadata = extract_exiftool_metadata(image_path)

    # Relative Altitude (usually in meters)
    rel_alt = float(metadata.get('RelativeAltitude', 0))

    # Gimbal Pitch (try 'GimbalPitchDegree' or fallback to 'GimbalPitch')
    pitch_deg = float(metadata.get('GimbalPitchDegree', metadata.get('GimbalPitch', -90)))

    # DateTimeOriginal (format "YYYY:MM:DD HH:MM:SS")
    datetime_str = metadata.get('DateTimeOriginal')
    if not datetime_str:
        raise ValueError("No DateTimeOriginal tag found in image EXIF metadata.")
    time_str = datetime_str.split(' ')[1]

    # FocalLength (usually in mm)
    focal_length_str = metadata.get('FocalLength', '0')
    focal_length_mm = float(focal_length_str.split(' ')[0])  # take only the numeric part before any space

    # Sensor width and height fallback defaults (in mm)
    sensor_width_mm = 17.3
    sensor_height_mm = 13.0

    # Image dimensions
    img_width = int(metadata.get('ImageWidth', 0))
    img_height = int(metadata.get('ImageHeight', 0))

    # GPS Coordinates
    gps_lat = metadata.get('GPSLatitude')
    gps_lon = metadata.get('GPSLongitude')

    def dms_to_decimal(dms_str):
        """
        Converts a DMS (Degrees, Minutes, Seconds) string to decimal degrees.
        Example input: '24 deg 13\' 36.24" S'
        """
        pattern = r'(\d+)[^\d]+(\d+)[^\d]+([\d.]+)[^\d]+([NSEW])'
        match = re.match(pattern, dms_str.strip())
        if not match:
            raise ValueError(f"Invalid DMS format: {dms_str}")
        degrees, minutes, seconds, direction = match.groups()
        decimal = float(degrees) + float(minutes)/60 + float(seconds)/3600
        if direction in ['S', 'W']:
            decimal = -decimal
        return decimal

    latitude = dms_to_decimal(gps_lat) if gps_lat else None
    longitude = dms_to_decimal(gps_lon) if gps_lon else None

    return rel_alt, pitch_deg, time_str, focal_length_mm, sensor_width_mm, sensor_height_mm, img_width, img_height, latitude, longitude

def extract_metadata(image_path):
    rel_alt, pitch_deg, _, focal_length_mm, sensor_width_mm, sensor_height_mm, img_width, img_height, lat, lon = extract_exif_metadata_partial(image_path)
    return lat, lon, rel_alt, pitch_deg, focal_length_mm, sensor_width_mm, sensor_height_mm, img_width, img_height

def get_camera_intrinsics(focal_length_mm, sensor_width_mm, sensor_height_mm, img_width, img_height):
    fx = img_width * focal_length_mm / sensor_width_mm
    fy = img_height * focal_length_mm / sensor_height_mm
    cx = img_width / 2
    cy = img_height / 2

    K = np.array([[fx, 0, cx],
                  [0, fy, cy],
                  [0, 0, 1]])
    return K

def get_rotation_matrix(pitch_deg, yaw_deg=0.0, roll_deg=0.0):
    pitch = math.radians(pitch_deg)
    yaw = math.radians(yaw_deg)
    roll = math.radians(roll_deg)

    R_x = np.array([[1, 0, 0],
                    [0, math.cos(roll), -math.sin(roll)],
                    [0, math.sin(roll), math.cos(roll)]])

    R_y = np.array([[math.cos(pitch), 0, math.sin(pitch)],
                    [0, 1, 0],
                    [-math.sin(pitch), 0, math.cos(pitch)]])

    R_z = np.array([[math.cos(yaw), -math.sin(yaw), 0],
                    [math.sin(yaw), math.cos(yaw), 0],
                    [0, 0, 1]])

    R = R_z @ R_y @ R_x
    return R

def enu_to_wgs84(x, y, origin_lat, origin_lon):
    transformer_local = Transformer.from_crs("epsg:4979", "epsg:4978", always_xy=True)
    transformer = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)
    z = 0
    ecef_x0, ecef_y0, ecef_z0 = transformer_local.transform(origin_lon, origin_lat, z)
    ecef_x = ecef_x0 + x
    ecef_y = ecef_y0 + y
    ecef_z = ecef_z0
    lon, lat, _ = transformer.transform(ecef_x, ecef_y, ecef_z)
    return lat, lon

def image_point_to_gps(u, v, K, R, cam_height, origin_lat, origin_lon):
    pixel = np.array([u, v, 1])
    ray_cam = np.linalg.inv(K) @ pixel
    ray_world = R @ ray_cam

    scale = -cam_height / ray_world[2]
    ground_point = ray_world * scale

    enu_x, enu_y = ground_point[0], ground_point[1]

    lat, lon = enu_to_wgs84(enu_x, enu_y, origin_lat, origin_lon)
    return lat, lon

def run_pipeline(image_path, bounding_boxes):
    lat, lon, alt, pitch_deg, focal_length_mm, sensor_width_mm, sensor_height_mm, img_width, img_height = extract_metadata(image_path)

    K = get_camera_intrinsics(focal_length_mm, sensor_width_mm, sensor_height_mm, img_width, img_height)
    R = get_rotation_matrix(pitch_deg)

    gps_results = []
    for i, box in enumerate(bounding_boxes, start=1):
        x_min, y_min, x_max, y_max = box
        center_u = (x_min + x_max) / 2 * img_width  # convert relative to pixels if needed
        center_v = (y_min + y_max) / 2 * img_height

        gps_lat, gps_lon = image_point_to_gps(center_u, center_v, K, R, alt, lat, lon)
        gps_results.append((i, (center_u, center_v), (gps_lat, gps_lon)))

    return gps_results

def plot_results(image_path, results):
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    for obj_id, (u, v), (lat, lon) in results:
        plt.plot(u, v, 'ro')
        plt.text(u + 5, v, f"ID {obj_id}: {lat:.5f}, {lon:.5f}", color='yellow', fontsize=8)
    plt.title("Bounding Box Centers with GPS Coordinates")
    plt.axis('off')
    plt.show()


In [26]:
def save_results_to_csv(results, csv_path):
    with open(csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ObjectID', 'Latitude', 'Longitude'])
        for obj_id, (u, v), (lat, lon) in results:
            writer.writerow([obj_id, f"{lat:.6f}", f"{lon:.6f}"])


def save_image_with_ids(image_path, results, output_path):
    img = cv2.imread(image_path)

    # Overlay only object IDs at the center points
    for obj_id, (u, v), _ in results:
        pos = (int(u), int(v))
        cv2.putText(img, f"ID {obj_id}", pos, cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1.5, color=(0, 255, 255), thickness=3, lineType=cv2.LINE_AA)

    cv2.imwrite(output_path, img)
    print(f"Saved image with IDs overlayed to {output_path}")

import os

def get_base_name(image_path):
    return os.path.splitext(os.path.basename(image_path))[0]

from geopy.distance import geodesic

def save_distances_csv(results, csv_path):
    # results: list of tuples (id, (u,v), (lat, lon))
    # Calculate pairwise distances in meters
    ids = [r[0] for r in results]
    coords = [(r[2][0], r[2][1]) for r in results]  # (lat, lon)

    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        header = ['ObjectID_1', 'ObjectID_2', 'Distance_meters']
        writer.writerow(header)

        for i in range(len(results)):
            for j in range(i+1, len(results)):
                dist = geodesic(coords[i], coords[j]).meters
                writer.writerow([ids[i], ids[j], f"{dist:.2f}"])

In [27]:
def load_bounding_boxes_from_txt(txt_path, conf_threshold=0.0):
    """
    Loads bounding boxes from a text file with lines formatted as:
    cls conf xmin_norm ymin_norm xmax_norm ymax_norm

    Args:
        txt_path (str): Path to the txt file.
        conf_threshold (float): Minimum confidence threshold to keep a bbox.

    Returns:
        List of tuples: (xmin_norm, ymin_norm, xmax_norm, ymax_norm)
    """
    bounding_boxes = []
    with open(txt_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 6:
                # Ignore malformed lines
                continue

            cls, conf, xmin, ymin, xmax, ymax = parts
            conf = float(conf)
            if conf < conf_threshold:
                continue

            xmin = float(xmin)
            ymin = float(ymin)
            xmax = float(xmax)
            ymax = float(ymax)

            bounding_boxes.append((xmin, ymin, xmax, ymax))
    return bounding_boxes

In [28]:
import time

def wait_for_file(file_path, timeout=30):
    """Waits for a file to exist up to a timeout (in seconds)."""
    start_time = time.time()
    while not os.path.exists(file_path):
        time.sleep(0.3)
        if time.time() - start_time > timeout:
            return False
    return True


In [29]:
import os
from pathlib import Path
import shutil

# --- Define shared paths ---
images_folder = "Images"

# Step 1: Run detection on all images
image_files = list(Path(images_folder).glob("*.JPG"))

for image_file in image_files:
    image_path = str(image_file)
    print(f"\nProcessing image: {image_path}")
    Find_Elephants(image_path)


Processing image: Images/DJI_0396.JPG

image 1/1 /content/drive/MyDrive/datasets/GPS/Images/DJI_0396.JPG: 480x640 4 Elephants, 3526.2ms
Speed: 12.7ms preprocess, 3526.2ms inference, 28.3ms postprocess per image at shape (1, 3, 480, 640)


In [31]:
# Create output folders if they don't exist
output_folder_Distances = "Processed_Distances_Output"
output_folder_Coordinates = "Processed_Coordinates_Output"
output_folder_Images = "Processed_Images_Output"
os.makedirs(output_folder_Distances, exist_ok=True)
os.makedirs(output_folder_Coordinates, exist_ok=True)
os.makedirs(output_folder_Images, exist_ok=True)

images_folder = "Images"

# Iterate over images in the Detections folder
for image_file in Path(images_folder).glob("*.JPG"):
    image_path = str(image_file)
    base_name = image_file.stem
    txt_file = os.path.join("Detections", f"{base_name}.txt")

    print(f"\nProcessing image: {image_path}")

    # Step 3: Load detections
    boxes = load_bounding_boxes_from_txt(txt_file, conf_threshold=0.6)

    # Step 4: Run full pipeline
    results = run_pipeline(image_path, boxes)

    # Step 5: Define output paths
    csv_output_path = os.path.join(output_folder_Coordinates, f"{base_name}_object_gps_coordinates.csv")
    image_output_path = os.path.join(output_folder_Images, f"{base_name}_output_with_ids.jpg")
    distance_csv_path = os.path.join(output_folder_Distances, f"{base_name}_object_distances.csv")

    # Step 6: Save outputs
    save_results_to_csv(results, csv_output_path)
    save_image_with_ids(image_path, results, image_output_path)
    save_distances_csv(results, distance_csv_path)

    print(f"Saved:\n- {csv_output_path}\n- {image_output_path}\n- {distance_csv_path}")



Processing image: Images/DJI_0396.JPG
Saved image with IDs overlayed to Processed_Images_Output/DJI_0396_output_with_ids.jpg
Saved:
- Processed_Coordinates_Output/DJI_0396_object_gps_coordinates.csv
- Processed_Images_Output/DJI_0396_output_with_ids.jpg
- Processed_Distances_Output/DJI_0396_object_distances.csv
